In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
from nltk.metrics import ConfusionMatrix
import numpy as np
from sklearn.externals import joblib

In [2]:
emo = 5

if emo == 2: 
    dataset_path = '../Data/Datasets/Binary Classification/'
else:
    dataset_path = '../Data/Datasets/Multiclass Classification/'

In [3]:
def convert_features(sentence):
    return {i:(i in sentence) for i in word_features}

In [4]:
#build the classifier
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

whole_ds = train.append(test,ignore_index=True )
whole_ds['tokenized_sents'] = whole_ds.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)

all_words = []
for line in whole_ds['tokenized_sents']:
    for w in line:
        all_words.append(w.lower())
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())
joblib.dump(word_features, '../Data/nb_word_features.pkl') 


featured_whole_ds = [(convert_features(row['tokenized_sents']), row[1]) for index, row in whole_ds.iterrows()]
classifier = nltk.NaiveBayesClassifier.train(featured_whole_ds)
joblib.dump(classifier, dataset_path+'nb_classifier.pkl') 

['../Data/Datasets/Multiclass Classification/nb_classifier.pkl']

In [ ]:
#hold out testing
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)
    

train['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in train.iterrows()]
featured_train = [(convert_features(row['tokenized_sents']), row[1]) for index, row in train.iterrows()]
test['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in test.iterrows()]
#test.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
featured_test = [(convert_features(row['tokenized_sents']), row[1]) for index, row in test.iterrows()]

classifier = nltk.NaiveBayesClassifier.train(featured_train)
score = nltk.classify.accuracy(classifier, featured_test)
print("Holdout testing accuracy: {}".format(score))

Holdout testing accuracy: 0.7812166488794023


In [ ]:
#Cross validation testing
accuracy_nb = []
folds = 10

for idx in range(1,folds+1):
    
    train = pd.read_csv(dataset_path+'cv'+str(idx)+'_train.csv', sep=",", header=None,index_col = False)
    test = pd.read_csv(dataset_path+'cv'+str(idx)+'_test.csv', sep=",", header=None,index_col = False)
    
    train['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in train.iterrows()]
    featured_train = [(convert_features(row['tokenized_sents']), row[1]) for index, row in train.iterrows()]
    test['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in test.iterrows()]
    featured_test = [(convert_features(row['tokenized_sents']), row[1]) for index, row in test.iterrows()]

    classifier = nltk.NaiveBayesClassifier.train(featured_train)
    score = nltk.classify.accuracy(classifier, featured_test)
    accuracy_nb.append(score)
    print("Fold: {}: accuracy = {}".format(idx,score))

print("10 fold Cross Validation accuracy = {}".format(np.mean(accuracy_nb)))

Fold: 1: accuracy = 0.8333333333333334
Fold: 2: accuracy = 0.7931034482758621
Fold: 3: accuracy = 0.7792553191489362
Fold: 4: accuracy = 0.7845744680851063
Fold: 5: accuracy = 0.7620320855614974
Fold: 6: accuracy = 0.7506702412868632
Fold: 7: accuracy = 0.7613941018766756
